In [1]:
# test for cuda and cudnn
import torch
print("version", torch.__version__)
print("cuda available", torch.cuda.is_available())
print("cudnn", torch.backends.cudnn.enabled)
print("device_name", torch.cuda.get_device_name(0))
print("device count", torch.cuda.device_count())
print("current dev", torch.cuda.current_device())
print("device zero", torch.cuda.device(0))


version 2.1.2
cuda available True
cudnn True
device_name NVIDIA GeForce GTX TITAN X
device count 4
current dev 0
device zero <torch.cuda.device object at 0x7fbcc8634be0>


In [8]:
from ocatari.core import OCAtari
import random

env = OCAtari("MsPacman-v4", mode="vision", hud=True, render_mode="rgb_array")
observation, info = env.reset()
env.objects

[Player at (78, 103), (9, 10),
 Ghost at (79, 57), (9, 10),
 Ghost at (79, 57), (9, 10),
 Ghost at (79, 57), (9, 10),
 Ghost at (79, 57), (9, 10)]

In [9]:
from ocatari.ram.extract_ram_info import detect_objects_raw, detect_objects_revised, init_objects, get_max_objects

objects = init_objects("MsPacman", True)
objects

[Player at (78, 103), (9, 10),
 Ghost at (79, 57), (9, 10),
 Ghost at (79, 57), (9, 10),
 Ghost at (79, 57), (9, 10),
 Ghost at (79, 57), (9, 10),
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]